In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown
import requests

In [ ]:
load_dotenv(override=True)

##### Search Agent

This agent will use the WebSearchTool provided by OpenAI SDK.  
OpenAi SDK provides a prompt which can be used for the best search results for the websearch we will use that

In [ ]:
instructions = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=instructions,
    tools=[WebSearchTool(search_context_size="low")], # size here determines the context of the search results the more the expensive
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required") # Required means that we will always use the tool when available
)

##### Structured output

We will ask an LLM to generate steps to perform a task.  
This tasks should be a finite number of searches. The more the better results but also will increase the costs.  
Steps:  
- Set number of searches.
- Write the instruction to generate those searches.
- Save them in a list (Structured Output)
    - Here we will use the Pydantic model for searches
- Write the agent to perfeom the searches and output the type as specified in the Structured output.

In [ ]:
no_of_searches = 3

instructions = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
    to perform to best answer the query. Output {no_of_searches} terms to query for."

# Pydantic models for Websearch
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this seach is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")
    
# Agent which will output the search terms to use
planner_agent = Agent(
    name="PlannerAgent",
    instructions=instructions,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)


##### Push Notification Setup

In [ ]:
# Setup Push notification
def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )
@function_tool
def send_push_notification(message : str):
    push(message)
    return {"status": "success"}

In [ ]:
# Agent to send the HTML formatted email
instructions = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provieded with a detailed report. You should use your tool to send one push notification, providing the report
converted into clean, well presented HTML with an appropriate subject line."""

notification_agent = Agent(
    name="notification_agent",
    tools=[send_push_notification],
    instructions=instructions,
    model="gpt-4o-mini",
)

##### Senior Research Agent

In [ ]:
instructions = (
    "You are a senior researcher tasked with writing a cohesive report for a research query."
    "You will be provided with the original query, and some initial research done by a research assistant \n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as a your final output. \n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short summary of the report, 2-3 sentences.")
    mardown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")
    
writer_agent = Agent(
    name="WriteAgent",
    instructions=instructions,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### Plan
Now comes the main heavy lifting of the multiagent flow.  
Recap:  
- Research assistant: Agent which will provide a nofluff reasearch of a particular topic using websearch tools.
- Planner Agent: Agent which will divide the research topic into different parts.
- Senior Research Agent(writer_agent): Agent who is responsible for generating the report.
    - Receives all the research from the assistants and consolidates them to genrate the final report.
- Notification Agent: Agent responsible for formatting and sending the final report to the user.
    

In [ ]:
# Create Plan
async def plan_searches(query: str):
    """ Uses the planner agent to plan which searched to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches.")
    return result.final_output

# Perform Individual searches helper function
async def search(item: WebSearchItem):
    """ Uses the search agent to perform a single search """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output
    
# perform searches in parallel
async def perform_searches(search_plan: WebSearchPlan):
    """ Uses the search agent to perform the searches in the search plan """
    print("Performing searches...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results


In [ ]:
async def write_report(query: str, search_results: list[str]):
    """ Uses the writer agent to write the report based on the query and search results """
    print("Thinking about the report...")
    input = f"Original Query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_notification(report: ReportData):
    """ Uses the notification agent to send a notification with the report """
    result = await Runner.run(notification_agent, report.mardown_report)
    print("Notification sent")
    return report

In [ ]:
query = "Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_notification(report)
    print("Research completed successfully.")